- 제너레이터 만들어서 날짜 뱉어내고, 하루 단위로 실행하도록 만들기.
- 제발 클래스로...
- retry 데코레이터 ㅇㅇㅇ
- HTTPError 429 : fakeuseragent 해결 가능한지 확인

- 함수 바꾼 부분 정리해놓기!!!!!

In [4]:
# install GOT3 package
!pip install GetOldTweets3
!pip install tqdm

In [1]:
# module import

import GetOldTweets3 as got
from bs4 import BeautifulSoup

import sys
import urllib
import json
from time import sleep

import datetime
import time
import os
from random import uniform
from tqdm import tqdm_notebook as tn
import csv

In [2]:
def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    """
    Invoke an HTTP query to Twitter.
    Should not be used as an API function. A static method.
    """
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''

    if hasattr(tweetCriteria, 'querySearch'):
        urlGetData += tweetCriteria.querySearch

    if hasattr(tweetCriteria, 'excludeWords'):
        urlGetData += ' -'.join([''] + tweetCriteria.excludeWords)

    if hasattr(tweetCriteria, 'username'):
        if not hasattr(tweetCriteria.username, '__iter__'):
            tweetCriteria.username = [tweetCriteria.username]

        usernames_ = [u.lstrip('@') for u in tweetCriteria.username if u]
        tweetCriteria.username = {u.lower() for u in usernames_ if u}

        usernames = [' from:'+u for u in sorted(tweetCriteria.username)]
        if usernames:
            urlGetData += ' OR'.join(usernames)

    if hasattr(tweetCriteria, 'within'):
        if hasattr(tweetCriteria, 'near'):
            urlGetData += ' near:"%s" within:%s' % (tweetCriteria.near, tweetCriteria.within)
        elif hasattr(tweetCriteria, 'lat') and hasattr(tweetCriteria, 'lon'):
            urlGetData += ' geocode:%f,%f,%s' % (tweetCriteria.lat, tweetCriteria.lon, tweetCriteria.within)

    if hasattr(tweetCriteria, 'since'):
        urlGetData += ' since:' + tweetCriteria.since

    if hasattr(tweetCriteria, 'until'):
        urlGetData += ' until:' + tweetCriteria.until

    if hasattr(tweetCriteria, 'minReplies'):
        urlGetData += ' min_replies:' + tweetCriteria.minReplies

    if hasattr(tweetCriteria, 'minFaves'):
        urlGetData += ' min_faves:' + tweetCriteria.minFaves

    if hasattr(tweetCriteria, 'minRetweets'):
        urlGetData += ' min_retweets:' + tweetCriteria.minRetweets

    if hasattr(tweetCriteria, 'lang'):
        urlLang = 'l=' + tweetCriteria.lang + '&'
    else:
        urlLang = ''
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers

    if debug:
        print(url)
        # print('\n'.join(h[0]+': '+h[1] for h in headers))

    # HTTP Request 429 : 타임 슬립 줘보자
    time.sleep(3)

    try:
        response = opener.open(url)
        jsonResponse = response.read()

    except TimeoutError as e:
        print("Timeout error")
        print('sleep 30')
        time.sleep(30)

        # 이 부분 나중에 retry로 진행하기
        try:
            response = opener.open(url)
            jsonResponse = response.read()
        except TimeoutError as e:
            print("Timeout Error again.")
            print("Pass Data!!!!!!!!!")
            pass

    except Exception as e:
        print("An error occured during an HTTP request:", str(e))
        print("Try to open in browser: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))

        # 이 부분 나중에 확인하기
        print("sleep 30!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        pass
        # sys.exit()
        

    try:
        s_json = jsonResponse.decode()
    except:
        print("Invalid response from Twitter")
        pass
        # sys.exit()
    else:
        try:
            dataJson = json.loads(s_json)
        except:
            print("Error parsing JSON: %s" % s_json)
        pass
        # sys.exit()

#     if debug:
#         print(s_json)
#         print("---\n")

    return dataJson

In [3]:
# 날짜 다시 입력해야 할 때 에러 발생시키자.
class NotValidEndDateError(Exception):
    def __init__(self):
        super().__init__('마지막 검색 날짜를 다시 설정하십시오.')

In [4]:
# GetOldTweets3 패키지의 setUntil 메소드는 마지막 날짜 포함하지 않으므로, 주의.
def set_crawl_date(start_date, end_date):
    
    start_date = datetime.datetime.strptime(str(start_date), "%Y%m%d")
    end_date = datetime.datetime.strptime(str(end_date), "%Y%m%d") - datetime.timedelta(days=1)
    
    if end_date == start_date:
        raise NotValidEndDateError
    
    else:   
        print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))    
        return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

In [5]:
def crawl_tweets(start_date, end_date):    
    got.manager.TweetManager.getJsonResponse = getJsonResponse # 함수에 이 부분 넣어주고
    
    print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
    start_time = time.time()
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=True)
    
    elapsed_time = time.time()-start_time
    
    print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    print("총 수집 트윗 개수 : {0}".format(len(tweets)))
    
    return tweets

In [6]:
def get_results(tweet_data):
    results = []
    for tweet in tn(tweet_data):
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
        
    return results        

In [7]:
def save_tweets(tweet_lists):
    base_file_dir = "tweets"

    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tn(tweet_lists):
            writer.writerow(list(tweet_list.values()))
            
    return 

In [55]:
crawl_start, crawl_end = set_crawl_date(20200208, 20200210)

트윗 수집 날짜 설정: 2020-02-08 00:00:00부터 2020-02-09 00:00:00까지


In [57]:
tweet_results = crawl_tweets(crawl_start, crawl_end)

========== 트윗 수집 시작: 2020-02-08 ~ 2020-02-09 ==========
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaEwLSphv3VhCIWgoCo0fyd1oQiEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLeVhd3VhCIWgoCo0fyd1oQiEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&ma

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLrF-vzOhCIWgoCo0fyd1oQiEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLvhvM3OhCIWgoCo0fyd1oQiEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLfJvJbOhCIWgoCo0fyd1oQiEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-08%20until%3A2020-02-09&src=typd&l=en&&include_available_features=1&include_entities=1&max_

KeyboardInterrupt: 

In [ ]:
tweet_results_lists = get_results(tweet_results)

In [ ]:
save_tweets(tweet_results_lists)

#### - 여러 날짜 크롤링 루프 처리

In [8]:
x = int(20200227)
y = int(20200229)

while True:
    
    if y == int(20200231):
        break
    
    else:
        crawl_start, crawl_end = set_crawl_date(x, y)
        tweet_results = crawl_tweets(crawl_start, crawl_end)
        tweet_results_lists = get_results(tweet_results)
        save_tweets(tweet_results_lists)
    
        print(f'========================= {x}-{y} : 저장완료!==============================')
    
        x = x+1
        y = y+1
    

트윗 수집 날짜 설정: 2020-02-27 00:00:00부터 2020-02-28 00:00:00까지
========== 트윗 수집 시작: 2020-02-27 ~ 2020-02-28 ==========
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLXl1_mQnSIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLWhm9mQnSIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLqVvL6InSIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaOwL69lJaInSIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCgKeR-fOHnSIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwL29_aH_nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAgLvVlt7-nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwKDB6Z3-nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaGwL2xsM30nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaGgLuFgrn0nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaGwLrdjK30nCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwKCdibXmnCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLWBq4rmnCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLS5ttflnCIWgICnrZmqkZ0iEjUAFQAlAFUAFQAA&reset_error_state=false
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-02-27%20until%3A2020-02-28&src=typd&l=en&&include_available_features=1&include_entities=1&max_

...

수집 완료 : 00:27:39
총 수집 트윗 개수 : 6774


C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


C:\Users\Jisoo\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.



========================= 20200227-20200229 : 저장완료!==============================


ValueError: day is out of range for month